#데이터 확인

In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("/content/food_orders_new_delhi (1) (2).csv")
data.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0


In [3]:
data.columns

Index(['Order ID', 'Customer ID', 'Restaurant ID', 'Order Date and Time',
       'Delivery Date and Time', 'Order Value', 'Delivery Fee',
       'Payment Method', 'Discounts and Offers', 'Commission Fee',
       'Payment Processing Fee', 'Refunds/Chargebacks'],
      dtype='object')

In [4]:
#주문 시간 datetime 형식으로 변환
data['Order Date and Time'] = pd.to_datetime(data['Order Date and Time'])

data.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0


#데이터 전처리

In [5]:
#discount 열 숫자만 뽑기 -> 배달앱 수익이면 필요없음
def extract_number(df):
  discount_str = df['Discounts and Offers']

  if not isinstance(discount_str, str): #nan 값 처리
        return 0

  if '%' in discount_str:
    percent = float(discount_str.split('%')[0])
    return df['Order Value'] * (percent/100)

  else:
      percent = float(discount_str.split(' ')[0])
      return df['Order Value'] * (percent/100)
data['Discount Amount'] = data.apply(extract_number, axis=1)

#-----------------------배달앱 전체 수익 계산 ------------------------------------------------------

#한 주문당 배달앱 전체 수익 계산 = 수수료 + 배달 수수료 + 결제 수수료
data['App Revenue'] = (data['Commission Fee'] +data['Delivery Fee'] +data['Payment Processing Fee'])

#-----------------------column 정리 ----------------------------------------------------------------

data = data.drop(columns = ['Order ID','Customer ID','Restaurant ID','Delivery Date and Time','Payment Method','Discounts and Offers'])

#----------------------환불된 상태 제거 (수익계산 어려움)---------------------------------------
cond = data['Refunds/Chargebacks'] ==0
data = data.loc[cond]

# 시간대 나누기

In [6]:
data.info() #order date and time 칼럼 데이터 타입 확인(datetime 형태여야함 !!)

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 998
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Order Date and Time     715 non-null    datetime64[ns]
 1   Order Value             715 non-null    int64         
 2   Delivery Fee            715 non-null    int64         
 3   Commission Fee          715 non-null    int64         
 4   Payment Processing Fee  715 non-null    int64         
 5   Refunds/Chargebacks     715 non-null    int64         
 6   Discount Amount         715 non-null    float64       
 7   App Revenue             715 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 50.3 KB


In [7]:
#새벽 00~08, 일과시간 08-18, 저녁 18-24
def day_split(time):
    hour = time.hour
    if hour < 8:
        return 'morning'
    elif hour < 18:
        return 'afternoon'
    else:
        return 'evening'

data['day'] = data['Order Date and Time'].apply(day_split)


In [8]:
#밑 시뮬레이션 코드 중 timedelta 이용해서 빼려고 시간만 따로 추출
data['Order Time'] = data['Order Date and Time'].dt.floor('H')


<ipython-input-8-81fdc41d859e>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data['Order Time'] = data['Order Date and Time'].dt.floor('H')


In [9]:
df1 = data[data['day']=='morning']
df2 = data[data['day']=='afternoon']
df3 = data[data['day']=='evening']

In [10]:
df = [df1,df2,df3] #함수 돌리기 위해서 리스트 형태로

# 시뮬레이션 함수

In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random


import numpy as np
import random
from datetime import timedelta

def simulation(df_list):
    simulation_minutes = 60
    simulation_count = 10000

    for i, df in enumerate(df_list, 1):  #
        start_time = df['Order Time'].min()
        end_time = df['Order Time'].max() - timedelta(minutes=simulation_minutes)

        random_start_times = [
            start_time + timedelta(seconds=random.randint(0, int((end_time - start_time).total_seconds())))
            for _ in range(simulation_count)
        ]

        simulation_losses = []
        for start in random_start_times:
            end = start + timedelta(minutes=simulation_minutes)
            mask = (df['Order Time'] >= start) & (df['Order Time'] < end)
            total_loss = df.loc[mask, 'App Revenue'].sum()
            simulation_losses.append(total_loss)

        average_loss = np.mean(simulation_losses)
        loss_per_minute = average_loss / simulation_minutes

      # 전체 평균 환율 (2024년 1~2월 평균)
        average_exchange_rate = (1334.74 + 1332.02) / 2  # = 1333.38

        print(f"--- [DataFrame {i}] ---")
        print(f"1시간 장애 시 평균 손해액: ${average_loss:,.2f}")
        print(f"분당 손해액 추정: ${loss_per_minute:,.2f}")

      # 원화 기준 계산 (2024년 1~2월 평균 환율 기준)
        average_loss_krw = average_loss * average_exchange_rate
        loss_per_minute_krw = loss_per_minute * average_exchange_rate

      # 출력
        print(f"2024년 1~2월 평균 환율 기준) 장애 시 평균 손해액: {average_loss_krw:,.0f}원")
        print(f"2024년 1~2월 평균 환율 기준) 분당 손해액 추정: {loss_per_minute_krw:,.0f}원")


In [18]:
simulation(df)

--- [DataFrame 1] ---
1시간 장애 시 평균 손해액: $49.32
분당 손해액 추정: $0.82
2024년 1~2월 평균 환율 기준) 장애 시 평균 손해액: 65,765원
2024년 1~2월 평균 환율 기준) 분당 손해액 추정: 1,096원
--- [DataFrame 2] ---
1시간 장애 시 평균 손해액: $60.74
분당 손해액 추정: $1.01
2024년 1~2월 평균 환율 기준) 장애 시 평균 손해액: 80,992원
2024년 1~2월 평균 환율 기준) 분당 손해액 추정: 1,350원
--- [DataFrame 3] ---
1시간 장애 시 평균 손해액: $38.73
분당 손해액 추정: $0.65
2024년 1~2월 평균 환율 기준) 장애 시 평균 손해액: 51,644원
2024년 1~2월 평균 환율 기준) 분당 손해액 추정: 861원


# 시간대별 차이 통계적으로 유의미한가?

In [24]:
import numpy as np
import random
from datetime import timedelta
from scipy.stats import f_oneway

def anova(df_list):
    simulation_minutes = 60
    simulation_count = 10000

    all_losses = []  # ANOVA를 위한 손해액 리스트들 저장

    for i, df in enumerate(df_list, 1):
        start_time = df['Order Time'].min()
        end_time = df['Order Time'].max() - timedelta(minutes=simulation_minutes)

        random_start_times = [
            start_time + timedelta(seconds=random.randint(0, int((end_time - start_time).total_seconds())))
            for _ in range(simulation_count)
        ]

        simulation_losses = []
        for start in random_start_times:
            end = start + timedelta(minutes=simulation_minutes)
            mask = (df['Order Time'] >= start) & (df['Order Time'] < end)
            total_loss = df.loc[mask, 'App Revenue'].sum()
            simulation_losses.append(total_loss)

        # 전체 리스트에 추가
        all_losses.append(simulation_losses)

        average_loss = np.mean(simulation_losses)
        loss_per_minute = average_loss / simulation_minutes

    # ANOVA 분석
    f_stat, p_value = f_oneway(*all_losses)
    print("--- [ANOVA 결과] ---")
    print(f"F-통계량: {f_stat:.2f}")
    print(f"p-값: {p_value:.4f}")

    if p_value < 0.05:
        print("그룹 간 손해액 차이가 통계적으로 유의미합니다.")
    else:
        print("그룹 간 손해액 차이가 통계적으로 유의미하지 않습니다.")


In [25]:
anova(df)

--- [ANOVA 결과] ---
F-통계량: 99.65
p-값: 0.0000
그룹 간 손해액 차이가 통계적으로 유의미합니다.
